<a href="https://colab.research.google.com/github/nguyetvo/Boston-Marathon/blob/master/Nguyet_Vo_Boston_Marathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.cluster import KMeans, MeanShift, SpectralClustering, AffinityPropagation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from scipy.spatial.distance import cdist

In [0]:
site = 'https://raw.githubusercontent.com/llimllib/bostonmarathon/master/results/2014/results.csv'
data = pd.read_csv(site, sep=',')

In [0]:
data.head()

In [0]:
print(data.shape)
print(data.shape[0]/4)

So our rows divide evenly into 4

In [0]:
data.columns

In [0]:
for column in data.columns:
    print(column)
    print(data[column].isna().value_counts())
    print()

In [0]:
data.ctz.unique()

In [0]:
data.state.unique()

Not sure if these are appropriate measures, but I plan to drop the "ctz" column, because most of its values are null, and convert all null values of the "state" column to 'unknown.'

In [0]:
# Drop the "ctz" column, replace nan in "state" column with 'unknown', replace '-' in the
# "_k" columns with 0
data.drop('ctz', axis=1, inplace=True)
data.replace(np.nan, 'unknown', inplace=True, regex=True)
data.replace('-', 0, inplace=True, regex=True)

In [0]:
# Setup X and y variables for modeling, drop name from predictors as it will not be helpful
X = data.drop(['overall', 'name'], axis=1)

# Normalize both target and predictors
X_norm = MinMaxScaler().fit_transform(X[['10k', 'division', '25k', 'age', 'official', 'genderdiv', '35k', 'pace', '30k', '5k', 'half', '20k', '40k']])
y_norm = MinMaxScaler().fit_transform(data[['overall']])

### Divide data into 4 even subsets to test for model consistency

In [0]:
# Perform PCA for graphic representation
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_norm)

# Create 4 equal data subsets to test our clustering and models on
X_1, X_2, X_pca1, X_pca2 = train_test_split(X_norm, X_pca, test_size=0.5, random_state=5588)

# Create final groupings
X1, X2, XPCA1, XPCA2 = train_test_split(X_1, X_pca1, test_size=0.5, random_state=5588)
X3, X4, XPCA3, XPCA4 = train_test_split(X_2, X_pca2, test_size=0.5, random_state=5588)

subsets = [X1, X2, X3, X4]

### Run KMeans consistency analysis with plotting

In [0]:
ypred = pd.DataFrame()

for counter, vals in enumerate([
    (X1, XPCA1),
    (X2, XPCA2),
    (X3, XPCA3),
    (X4, XPCA4)]):
    
    # Put the features into ypred.
    ypred['pca_f1' + '_sample' + str(counter)] = vals[1][:, 0]
    ypred['pca_f2' + '_sample' + str(counter)] = vals[1][:, 1]
    
    # Generate cluster predictions and store them for clusters 2 to 6.
    for nclust in range(2, 7):
        pred = KMeans(n_clusters=nclust, random_state=5588).fit_predict(vals[0])
        ypred['clust' + str(nclust) + '_sample' + str(counter)] = pred

In [0]:
# For each  number of clusters, plot the clusters using the
# pca features for each sample.
for cluster in range(2, 7):
    
    # Make a grid of subplots.
    f, axarr = plt.subplots(2, 2)
    
    # Make a plot for each sample.
    for i in range(4):
        
        # PCA-created features.
        x_sub = ypred['pca_f1_sample{}'.format(i)]
        y_sub = ypred['pca_f2_sample{}'.format(i)]
        
        # Cluster assignments.
        c = ypred['clust{}_sample{}'.format(cluster, i)]
        
        # Assign the subplot to its place on the grid.
        rows = int(np.floor(i / 2))
        cols = i % 2
        axarr[rows, cols].scatter(x_sub, y_sub, c=c)
        axarr[rows, cols].set_title('sample {}'.format(i))
        axarr[rows, cols].set_xlim([-1, 1])
        axarr[rows, cols].set_ylim([-1, 1])
    
    # Space out the plots so that the headings don't overlap axis values.
    plt.suptitle('{} Clusters'.format(cluster), fontsize=20)
    plt.tight_layout()
    plt.show()
    print('\n')

It appears to me that 3 clusters provides the best balance of number of clusters and consistency across data subsets. 4 clusters is also pretty good. Once you get above 4 clusters, boundaries start to shift a lot more.

### I don't see any means of measuring ARI on this data set, as we are using a numerical variable, so I will move on to assessing the Silhouette Coefficient using KMeans

In [0]:
for cluster in range(2, 7 ):
    print()
    print("{} clusters:".format(cluster))
    for sample in subsets:
        model = KMeans(n_clusters=cluster, random_state=42).fit(sample)
        labels = model.labels_
        print(metrics.silhouette_score(sample, labels, metric='euclidean'))

### Looks like the 3 cluster model wins just slightly over the 2 cluster. The scores are all pretty consistent across the 4 subsets of the data as well, which is positive.

### KMeans Elbow Method

In [0]:
# Loop through all 4 subsets of the data
it = 1
K = range(1, 10)
for X in subsets:
    # Determine distortion for each value of k clusters
    distortions = []
    for k in K:
        k_mean_model = KMeans(n_clusters=k).fit(X)
        k_mean_model.fit(X)  # Why do we need to fit X again after we already assign a fit model?
        distortions.append(np.sum(np.min(cdist(X, k_mean_model.cluster_centers_, 'euclidean'),
                                         axis=1))/X.shape[0])
    
    # Create coordinates for line segment to plot start point to end point
    x1, y1 = 1, distortions[0]
    x2, y2 = 9, distortions[-1]
    
    # Plot elbow
    plt.plot(K, distortions, 'bx-')
    plt.plot((x1, x2), (y1, y2))
    plt.title('Elbow Method plot for data subset {}'.format(it))
    plt.xlabel('# Clusters')
    plt.ylabel('Distortion')
    plt.show()
    
    # Determing optimal cluster size, best balance between within-cluster homogeneity and 
    # cluster diversity by finding point furthest from straight line (x1, y1), (x2, y2)
    distances = []
    for i in range(len(distortions)):
        x0 = i + 1
        y0 = distortions[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2* x1)
        denominator = ((y2 - y1)**2 + (x2 - x1)**2)**0.5
        distances.append(numerator/denominator)
    # Return best n_clusters value
    print('Optimal number of clusters:', distances.index(max(distances)) + 1)
    it += 1

We have undecided elbow method plots between the 4 subsets of data. Based on all the methods together as well as the plotting above, I would say 3 clusters would be best. The Silhouette Coefficient tipped the scales.

## Create KMeans model with ideal n_clusters and evaluate entired data set

In [0]:
import itertools

y_norm = np.array(list(itertools.chain.from_iterable(y_norm)))

In [0]:
best_kmean = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10,
                    tol=0.0001, n_jobs=-1, random_state=5588)
y_pred = best_kmean.fit_predict(X_norm)

# Plot the solution.
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred)
plt.show()

# Check the solution against the data.
print('Comparing k-means clusters against the data:')
print(pd.crosstab(y_pred, y_norm))

In [0]:
data['cluster'] = y_pred + 1

In [0]:
data.head()

### Run Times (5k, 10k, 20k, half, 25k, 30k, 35k, 40k, official)

In [0]:
# Need all of our time columns to be numeric not objects
dist_col = ['5k', '10k', '20k', 'half', '25k', '30k', '35k', '40k', 'official']
for column in dist_col:
    data[column] = data[column].astype(float)

In [0]:
plt.figure(figsize=(12, 15))

for dist in dist_col:
    plt.subplot(3, 3, dist_col.index(dist)+1)
    sns.boxplot(x='cluster', y=dist, data=data)
    plt.title('{} times by cluster number'.format(dist))
    
plt.tight_layout()
plt.show()

The configuration of the boxplots remains impressively consistent across all measures of distance. Runners in cluster 3 tend to be much faster than cluster 1 or 2. Cluster 2 tends to be a bit faster than cluster 1. 

Next we'll look at gender, starting with the distribution for each cluster.

### Gender

In [0]:
# need to find percentage of men and women per cluster.
gender = pd.DataFrame(columns=['F', 'M'])
gender.loc['cluster_1'] = data[data['cluster']==1].groupby('gender').count()['cluster']/data[data['cluster']==1].count()['cluster']
gender.loc['cluster_2'] = data[data['cluster']==2].groupby('gender').count()['cluster']/data[data['cluster']==2].count()['cluster']
gender.loc['cluster_3'] = data[data['cluster']==3].groupby('gender').count()['cluster']/data[data['cluster']==3].count()['cluster']

In [0]:
plt.subplots(1, 2, figsize=(10, 5))

# Stacked horizontal bar chart of percentages
plt.subplot(1, 2, 1)
plt.barh(gender.index, gender.F, color='c', label='Female')
plt.barh(gender.index, gender.M, left=gender.F, color='r', label='Male')
plt.title('Gender distribution in each cluster')
plt.ylabel('Cluster')
plt.legend()

# Countplot of clusters
plt.subplot(1, 2, 2)
sns.countplot(y='cluster', data=data, order=[3, 2, 1])
plt.title("Countplot of each cluster")
plt.tight_layout()
plt.show()

Interesting results here. We can see there are many more marathon participants who are male - clusters 2 and 3, both predominantly male, have the higher member counts. Female participants are only the majority in cluster 1, which has the lowest member count of the 3.

I'm also curious to see what the run times plots we created above look like when the two genders are separated.

In [0]:
plt.figure(figsize=(12, 15))

for dist in dist_col:
    plt.subplot(3, 3, dist_col.index(dist)+1)
    sns.boxplot(x='cluster', y=dist, hue='gender', data=data)
    plt.title('{} times by cluster number'.format(dist))
    
plt.tight_layout()
plt.show()

Here we see that male runners have consistently lower median times in all distance groupings than female runners for each of the 3 clusters. There is a greater gender discrepancy in cluster 3 than 1 or 2. And the IQR is shorter for cluster 3.

We still haven't found anything that greatly differentiates clusters 1 and 2. Hopefully we'll find something next, as we continue our bivariate plots with the 'cluster' feature and the remaining columns.

In [0]:
plt.figure(figsize=(12, 15))

plt.subplot(3, 2, 1)
sns.boxplot(x='cluster', y='age', data=data)
plt.title('Median age for each cluster')

plt.subplot(3, 2, 2)
sns.boxplot(x='cluster', y='pace', data=data)
plt.title('Cluster pace')

plt.subplot(3, 2, 3)
sns.boxplot(x='cluster', y='division', data=data)
plt.title('Cluster division')

plt.subplot(3, 2, 4)
sns.boxplot(x='cluster', y='genderdiv', data=data)
plt.title('Cluster gender division')

plt.subplot(3, 2, 5)
sns.boxplot(x='cluster', y='overall', data=data)
plt.title('Overall ranking by cluster and gender')

plt.tight_layout()
plt.show()


### Age
Cluster 1 represents participants under 40, cluster 2 is over 40, including outliers that stretch above 76/77 years old. Cluster 3 seems to run the entire range of ages but doesn't reach quite as high as cluster 2, having a few outliers as well going into the mid-70's.

### Pace
Pace reflects the same trend as the run times for each distance, as we would expect, being a measure of time taken to run a given distance. Cluster 3 is the lowest time, clusters 1 and 2 are very close, but 2 has a slightly lower median value. The boxplots have many outliers, all three have some above, cluster 3 is the only with outliers below the IQR.

### Division
The measure of a runner's rank in their division. Cluster 3 holds the lowest values or highest median division ranks, but cluster 2 is also very close. Here is another clear differentiation of clusters 1 and 2: 2 tends toward better division rank. Cluster 3 has a few outliers above the IQR.

### Gender Division
The measure of a runner's rank in their gender division. The gender division rank is pretty stable between clusters 1 and 2, but cluster 3 averages much lower values/higher ranks. Cluster 3 has a few outliers above the IQR.

### Overall
Overall rank of racers for the clusters follows the same trend as times: cluster 3 has the lowest values (highest ranks), clusters 1 and 2 are a bit higher up on the plot, the latter having just a slightly lower median

### State

In [0]:
states = pd.DataFrame()
states['state'] = sorted(data.state.unique())
states['cluster_1'] = np.array(data[data['cluster']==0].groupby('state').count()['cluster'] / data.groupby('state').count()['cluster'])
states['cluster_2'] = np.array(data[data['cluster']==1].groupby('state').count()['cluster'] / data.groupby('state').count()['cluster'])
states['cluster_3'] = np.array(data[data['cluster']==2].groupby('state').count()['cluster'] / data.groupby('state').count()['cluster'])
states.fillna(0, inplace=True)
states.head()

In [0]:
plt.figure(figsize=(10, 20))

# Create cluster_1 bars
plt.barh(states.state, states.cluster_1, color='#b5ffb9', edgecolor='white', label='Cluster 1')
# Create cluster_2 bars
plt.barh(states.state, states.cluster_2, left=list(states.cluster_1), color='#f9bc86', 
         edgecolor='white', label='Cluster 2')
# Create cluster_3 bars
plt.barh(states.state, states.cluster_3, left=[i+j for i,j in zip(states.cluster_1, 
        states.cluster_2)], color='#a3acff', edgecolor='white', label='Cluster 3')
 
plt.yticks(range(len(states.index)), states.state)
plt.ylabel("State")
plt.title('Percent of participants in each cluster by state')
plt.legend()
plt.show()

From the above plot, there's one interesting observation that stands out: the vast majority of participants in each state fall into cluster 3, the fastest group. However, Massachusetts and its most proximate two states, New Hampshire and Rhode Island, saw much higher rates of participation in clusters 1 and 2. 

I believe the story behind this is: people who are travelling from out of state to participate are going to be very serious about running. The bar to entry for local Bostonians and Massachusettsans is much lower, so many casual runners and such may join as a fun local activity. It's an interesting distinction the model makes, especially as I did not feed the 'state' column to my clustering model.

There are a few outlier "states," in which all points are only in one cluster. I have a feeling that these are caused by small sample sizes from these states, so I want to check into that:

In [0]:
data['state'].value_counts(ascending=True).head(10)

That pretty much sums up all the standout bars on the plot that didn't follow the typical distribution. The four bars with only one cluster represented were made up of fewer than 3 participants each! 


### Country

In [0]:
countries = pd.DataFrame()
countries['country'] = sorted(data.country.unique())
countries['cluster_1'] = np.array(data[data['cluster']==1].groupby('country').count()['cluster'] / data.groupby('country').count()['cluster'])
countries['cluster_2'] = np.array(data[data['cluster']==2].groupby('country').count()['cluster'] / data.groupby('country').count()['cluster'])
countries['cluster_3'] = np.array(data[data['cluster']==3].groupby('country').count()['cluster'] / data.groupby('country').count()['cluster'])
countries.fillna(0, inplace=True)
countries.head()

In [0]:
plt.figure(figsize=(10, 20))

# Create cluster_1 bars
plt.barh(countries.country, countries.cluster_1, color='#b5ffb9', edgecolor='white', 
         label='Cluster 1')
# Create cluster_2 bars
plt.barh(countries.country, countries.cluster_2, left=list(countries.cluster_1), 
         color='#f9bc86', edgecolor='white', label='Cluster 2')
# Create cluster_3 bars
plt.barh(countries.country, countries.cluster_3, left=[i+j for i,j in zip(countries.cluster_1, 
        countries.cluster_2)], color='#a3acff', edgecolor='white', label='Cluster 3')
 
plt.yticks(range(len(countries.index)), countries.country)
plt.ylabel("Country")
plt.title('Percent of participants in each cluster by country')
plt.legend()
plt.show()

There is a much wider variety of distributions in the countries plot than the US states plot we looked at before this. Cluster 3 obviously has the highest percentage of global participants, but there are many countries with only participants in clusters 1 or 2. On average, cluster 1 has the least participants, probably because it's expensive to travel and at a younger age, people are likely to have less money. 

For the countries with only values in one cluster, I'm curious to see if that can be explained by runner counts:

In [0]:
data['country'].value_counts(ascending=True).head(25)

Above, the majority of the one-cluster countries in the plot above are represented in the above 25 country list. That helps to make some sense of the variety in the cluster distributions for countries plot and assuage some of the worry of extreme outliers.

## 3-cluster analysis

The clusters our model created are indeed interesting. I'll do my best to sum up what characterizes each. Overall, it can be said that the run times for male runners were consistently shorter than for female runners. Otherwise:

__Cluster 1__: Characterized by higher marathon run times for all measured distances. This is a younger group, from 18-40 years of age, that is 60% female. As expected, if their times are higher, their pace and division spots also tend to be higher than their counterparts. There are fewer participants in cluster 1 for almost every country and state. One notable exception is Massachusetts, the event location, and I explained earlier my reasoning for that. I think of cluster 1 as young, casual runners, mostly from the Boston/Massachusetts area.

__Cluster 2__: Characterized by slightly lower marathon run times than cluster 1 for all measured distances, and around 60% male. It's almost as if clusters 1 and 2 are two halves of the same coin, cluster 1 is participants below 40, cluster 2 is participants above 40. We see one notable difference between the two clusters in the 'division' ranking, where cluster 2 is closer to cluster 3, averaging much higher ranks than cluster 1. As far as the 'state' and 'country' columns, we see a greater percentage of participants from cluster 2 than cluster 1, but not as many as cluster 3. As I mentioned above, this may have to do with even the most serious older runners averaging a slower pace than their serious young counterparts. Or it could be free time (retirement) and greater resources than young people who, most likely, are raising children or paying student debt, buying houses, etc. Cluster 2 could be summed up as older runners, mostly male, either competitive or casual, that is undecided.

__Cluster 3__: I would call cluster 3 the competitive runners from around the world. The times for each distance are shorter by a considerable amount than the other two clusters, and the pace, division ranking, gender division ranking, and overall ranking reflect that. Additionally, we see that most participants from states other than Massachusetts and other countries fall into cluster 3 - the majority of people who travel for this marathon are serious runners.

### Building a 4-cluster model:

In [0]:
best_kmean4 = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10,
                    tol=0.0001, random_state=5588)#, n_jobs=-1)
y_pred = best_kmean4.fit_predict(X_norm)

# Plot the solution.
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred)
plt.show()

# Check the solution against the data.
print('Comparing k-means clusters against the data:')
print(pd.crosstab(y_pred, y_norm))

In [0]:
data['4cluster'] = y_pred + 1

In [0]:
plt.figure(figsize=(12, 15))

for dist in dist_col:
    plt.subplot(3, 3, dist_col.index(dist)+1)
    sns.boxplot(x='4cluster', y=dist, data=data)
    plt.title('{} times by cluster number'.format(dist))
    
plt.tight_layout()
plt.show()

Here, cluster 2 contains the fastest runners for all distances. It appears that clusters 1 and 4 are most similar to 1 and 2 from our 3 cluster analysis. Cluster 3 here seems to be a new addition: faster than 1 or 4, but not the fastest.

### Gender

In [0]:
# need to find percentage of men and women per cluster.
gender = pd.DataFrame(columns=['F', 'M'])
gender.loc['cluster_1'] = data[data['4cluster']==1].groupby('gender').count()['4cluster']/data[data['4cluster']==1].count()['4cluster']
gender.loc['cluster_2'] = data[data['4cluster']==2].groupby('gender').count()['4cluster']/data[data['4cluster']==2].count()['4cluster']
gender.loc['cluster_3'] = data[data['4cluster']==3].groupby('gender').count()['4cluster']/data[data['4cluster']==3].count()['4cluster']
gender.loc['cluster_4'] = data[data['4cluster']==4].groupby('gender').count()['4cluster']/data[data['4cluster']==4].count()['4cluster']

In [0]:
plt.subplots(1, 2, figsize=(10, 5))

# Stacked horizontal bar chart of percentages
plt.subplot(1, 2, 1)
plt.barh(gender.index, gender.F, color='c', label='Female')
plt.barh(gender.index, gender.M, left=gender.F, color='r', label='Male')
plt.title('Gender distribution in each cluster')
plt.ylabel('Cluster')
plt.legend()

# Countplot of clusters
plt.subplot(1, 2, 2)
sns.countplot(y='4cluster', data=data, order=[4, 3, 2, 1])
plt.title("Countplot of each cluster")
plt.tight_layout()
plt.show()

Here we have a pretty similar breakdown of the clusters by gender. Again we have one smaller cluster (4) that is majority female, while the rest are majority male. We have one small (1) and two large (2, 3) majority male clusters.

### Other columns:

In [0]:
plt.figure(figsize=(12, 15))

plt.subplot(3, 2, 1)
sns.boxplot(x='4cluster', y='age', data=data)
plt.xlabel('Cluster')
plt.title('Median age for each cluster')

plt.subplot(3, 2, 2)
sns.boxplot(x='4cluster', y='pace', data=data)
plt.xlabel('Cluster')
plt.title('Cluster pace')

plt.subplot(3, 2, 3)
sns.boxplot(x='4cluster', y='division', data=data)
plt.xlabel('Cluster')
plt.title('Cluster division')

plt.subplot(3, 2, 4)
sns.boxplot(x='4cluster', y='genderdiv', data=data)
plt.xlabel('Cluster')
plt.title('Cluster gender division')

plt.subplot(3, 2, 5)
sns.boxplot(x='4cluster', y='overall', data=data)
plt.xlabel('Cluster')
plt.title('Overall ranking by cluster')

plt.tight_layout()
plt.show()


### Age
For age, we can see two clusters tend toward younger participants, two tend toward older participants. Clusters 1 and 4 seem to share a similar relatiomship to clusters 1 and 2 from our 3 cluster analysis: in cluster 4 all participants are below 40 years of age, and in cluster 1 they are all above 40 years of age. Cluster 2 tends to be younger than cluster 3 by about 15 years. So we have an older fast group (cluster 3), which is not as fast as our younger fast group (cluster 2).

### Cluster Pace
Pace again reflects the distribution from the distance subplots before this. Clusters 1 and 4 have the slowest paces, followed by cluster 3, then cluster 2 at the fastes paces.

### Cluster Division
This distribution looks a bit similar to our 3 cluster model. The division rankings don't seem to fall in line with the runner's time/speed so much. The median for cluster 2 is slightly lower than cluster 3, though, which represents our overall trend. Cluster 4 has the highest median division ranking. 

### Cluster Gender Division
Gender Division follows the same distribution as the distances and pace: Cluster 2 in the lead, cluster 3 behind that, clusters 1 and 4 are the highest(slowest).

### Overall Rank
Overall ranking also follows the above distribution (cluster gender division), no surprises.

### State

In [0]:
states4 = pd.DataFrame()
states4['state'] = sorted(data.state.unique())
states4['cluster_1'] = np.array(data[data['4cluster']==1].groupby('state').count()['4cluster'] / data.groupby('state').count()['4cluster'])
states4['cluster_2'] = np.array(data[data['4cluster']==2].groupby('state').count()['4cluster'] / data.groupby('state').count()['4cluster'])
states4['cluster_3'] = np.array(data[data['4cluster']==3].groupby('state').count()['4cluster'] / data.groupby('state').count()['4cluster'])
states4['cluster_4'] = np.array(data[data['4cluster']==4].groupby('state').count()['4cluster'] / data.groupby('state').count()['4cluster'])
states4.fillna(0, inplace=True)
states4.head()

In [0]:
plt.figure(figsize=(10, 20))

# Create cluster_1 bars
plt.barh(states4.state, states4.cluster_1, color='#b5ffb9', edgecolor='white', label='Cluster 1')
# Create cluster_2 bars
plt.barh(states4.state, states4.cluster_2, left=list(states4.cluster_1), color='#f9bc86', 
        edgecolor='white', label='Cluster 2')
# Create cluster_3 bars
plt.barh(states4.state, states4.cluster_3, left=[i+j for i,j in zip(states4.cluster_1, 
        states4.cluster_2)], color='#a3acff', edgecolor='white', label='Cluster 3')
# Create cluster_4 bars
plt.barh(states4.state, states4.cluster_4, left=[i+j+k for i,j,k in zip(states4.cluster_1, 
        states4.cluster_2, states4.cluster_3)], color='#7958b3', edgecolor='white', 
        label='Cluster 3')
    
plt.yticks(range(len(states4.index)), states4.state)
plt.ylabel("State")
plt.title('Percent of participants in each cluster by state')
plt.legend()
plt.show()

We have fairly consistent distribution of the four clusters across different states/territories. The outlier states with only one or two clusters are the same as last time - only 5 states/territories that qualify. We see that most of the participants from each state fall into clusters 2 and 3, our two fastest clusters. Again, Massachusetts has higher ratios of slower/less competitive runners (clusters 1 and 4), as do neighboring states New Hampshire (NH) and Maine (MN).

### Country

In [0]:
countries4 = pd.DataFrame()
countries4['country'] = sorted(data.country.unique())
countries4['cluster_1'] = np.array(data[data['4cluster']==1].groupby('country').count()['4cluster'] / data.groupby('country').count()['4cluster'])
countries4['cluster_2'] = np.array(data[data['4cluster']==2].groupby('country').count()['4cluster'] / data.groupby('country').count()['4cluster'])
countries4['cluster_3'] = np.array(data[data['4cluster']==3].groupby('country').count()['4cluster'] / data.groupby('country').count()['4cluster'])
countries4['cluster_4'] = np.array(data[data['4cluster']==4].groupby('country').count()['4cluster'] / data.groupby('country').count()['4cluster'])
countries4.fillna(0, inplace=True)
countries4.head()

In [0]:
plt.figure(figsize=(10, 20))

# Create cluster_1 bars
plt.barh(countries4.country, countries4.cluster_1, color='#b5ffb9', edgecolor='white', 
         label='Cluster 1')
# Create cluster_2 bars
plt.barh(countries4.country, countries4.cluster_2, left=list(countries4.cluster_1), 
         color='#f9bc86', edgecolor='white', label='Cluster 2')
# Create cluster_3 bars
plt.barh(countries4.country, countries4.cluster_3, left=[i+j for i,j in zip(countries4.cluster_1, 
         countries4.cluster_2)], color='#a3acff', edgecolor='white', label='Cluster 3')
# Create cluster_4 bars
plt.barh(countries4.country, countries4.cluster_4, 
         left=[i+j+k for i,j,k in zip(countries4.cluster_1, countries4.cluster_2, 
         countries4.cluster_3)], color='#7958b3', edgecolor='white', label='Cluster 3')

plt.yticks(range(len(countries4.index)), countries4.country)
plt.ylabel("Country")
plt.title('Percent of participants in each cluster by country')
plt.legend()
plt.show()

With the countries we also see a similar distribution of clusters as we did with the 3 cluster model and the states. Clusters 2 and 3, which are characterized by lower times/faster runners, make up the majority of the participants from around the world. Again, this is sensible. Clusters 1 and 4 represent a much smaller proportion of the runners as they are both made up of slower runners. 

There are just over 20 bars represented by only one or two clusters, most of which are on the list we saw earlier when we did the 3 cluster analysis: most are just very small pools of participants.

## Final 4 Cluster Analysis

We saw some similar distributions among clusters in our 3 and 4 cluster models, but it's not so simple as splitting one of the old clusters into two new ones. There was some shuffling about of runners into different clusters, so we'll have to reassess what makes each cluster unique:

__Cluster 1__: The smallest cluster in terms of number of rows, cluster 1 is over 60% male. These runners are the oldest group, with everyone being over 40 years old. The slowest runners (longest times) are members of this cluster and cluster 4. While the median run times are slightly higher for this cluster than cluster 4, the division rankings are much lower in cluster 1. But, the gender division rankings are the highest (worst). 

About 15% of US/Canadian participants are in cluster 1. Many countries send a high percentage of runners who are in cluster 1 (around 30%). Looking closer, some are represented by only a few runners, and I am inferring the many of the other countries tend to have higher GDP or middle/upper class populations. This includes many European countries (FRA, ITA, GER, DEN, POL, etc.), developed East Asian countries (JPN, KOR), and quickly developing Asian countries (CHN, IND).

We can say this is a smaller, older, slower cluster that is mostly male. It represents many runners from Massachusetts. These may be more casual runners as they aren't as fast.

__Cluster 2__: This cluster is the largest in number of members and the fastest (lowest times). The age range is quite wide, but does not contain the oldest participants (IQR stops at 60). Gender is the most balanced of any of the clusters, about 44% female/56% male. The largest number of participants from most states and most countries come from cluster 2. This cluster is more serious runners (which, understandably) makes up the largest group in an internationally renown marathon.

__Cluster 3__: Cluster 3 is the second largest and contains the second-fastest group of runners. The age range is pretty spread out, but all above 30 and extends all the way up to some of the oldest. Pace, gender division, and overall ranking all represent the same trend as the run times: above cluster 2, below clusters 1 and 4. The regular division ranking is slightly higher than cluster 2, slightly lower than cluster 1, and much lower than cluster 4. Cluster 3 is the second most represented cluster for almost all states. Many countries also show a large proportion of cluster 3 runners, but there is much more variance between countries. Overall, this cluster is a large group of serious runners who are slightly older than cluster 2. I would say those two clusters represent more passionate/fast runners, and the greatest distinction is age (cluster 2 being younger, on average).

__Cluster 4__: The final cluster is the only one that is majority female. It's a smaller cluster, and tends toward longer times, but not as long as cluster 1. This is the youngest cluster, with all participants below 40. This cluster has the highest gender division values (which may be related to its unique gender distribution). Otherwise its pace, division score, and overall ranking follow the same pattern as its times. For most states, cluster 4 is the least represented, Massachusetts being the largest exception. There are very few international runners in cluster 4, with some exceptions. Cluster 4 represents young, mostly female, casual/amature runners, many of whom are from Massachusetts/Boston area. 



**Add more functions**

In [0]:
# Initialize data frames
ypred = pd.DataFrame()
score = pd.DataFrame(columns=['cluster_pred','sil_score'])

# Keep track of counts of the models and use data from the different folds
for counter, data in enumerate([
    (X1, XPCA1),
    (X2, XPCA2),
    (X3, XPCA3),
    (X4, XPCA4)]):
    
    # Put the features into ypred.
    ypred['pca_f1' + '_sample' + str(counter)] = data[1][:, 0]
    ypred['pca_f2' + '_sample' + str(counter)] = data[1][:, 1]
    
    # Creating a list of possible number of clusters to test in kmeans.
    for nclust in range(2, 6):
       
        # Instantiating and fit_predicting model to then add to data frame
        kmeans = KMeans(n_clusters=nclust, random_state=42)
        pred = kmeans.fit_predict(data[0])
        ypred['clust' + str(nclust) + '_sample' + str(counter)] = pred
        
        # Calculating silhouette scores for the data and adding that to the shilouette score
        labels = kmeans.labels_
        sscore = metrics.silhouette_score(data[0], labels, metric='euclidean')
        score = score.append({'cluster_pred':'clust' + str(nclust) + '_sample' + str(counter), 
                              'silhouette_score':sscore}, ignore_index=True)

# Sorting sihoilette scores
score.sort_values(by='silhouette_score', ascending=False)

A five-cluster system has the highest silhouette scores

In [0]:
# For each  number of clusters, plot the clusters using the
# pca features for each sample.
for cluster in range(2, 6):
    
    # Make a grid of subplots.
    f, axarr = plt.subplots(2, 2)
    
    # Make a plot for each sample.
    for i in range(4):
        
        # PCA-created features.
        x_sub = ypred['pca_f1_sample{}'.format(i)]
        y_sub = ypred['pca_f2_sample{}'.format(i)]
        
        # Cluster assignments.
        c = ypred['clust{}_sample{}'.format(cluster, i)]
        
        # Assign the subplot to its place on the grid.
        rows = int(np.floor(i / 2))
        cols = i % 2
        axarr[rows, cols].scatter(x_sub, y_sub, c=c)
        axarr[rows, cols].set_title('sample {}'.format(i))
        axarr[rows, cols].set_xlim([-.5, .5])
        axarr[rows, cols].set_ylim([-.5, 1.2])
    
    # Space out the plots so that the headings don't overlap axis values.
    plt.suptitle('{} Clusters'.format(cluster), fontsize=20)
    plt.tight_layout()
    plt.show()
    print('\n')

**Mean-shift**

For a mean-shift model, we will use a range of quantiles to create bandwidths from 0.1 to 0.4, calculating the Silhouette scores for each.

In [0]:
# Model imports
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation

# Initialize new data frames
ypred_ms = pd.DataFrame()
score_ms = pd.DataFrame(columns=['cluster_pred','mean_shift', 'quantile'])

# Keep track of counts of the models and use data from the different folds
for counter, data in enumerate([X1, X2, X3, X4]):
    # Creating a list of possible quantiles to test in mean shift.
    for n in [0.1, 0.2, 0.3, 0.4]:
        # Estimating number of clusters for data
        bandwidth = estimate_bandwidth(data, quantile=n, n_samples=500)
        # Ensuring all sets are the same lenght
        data = data[:4013][:]
        # Instantiating and fit_predicting model to then add to data frame
        ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
        pred = ms.fit_predict(data)
        labels = ms.labels_
        cntrs = len(np.unique(labels))
        ypred_ms['clust' + str(cntrs) + '_sample' + str(counter)] = pred
        # Calculating silhouette scores for the data and adding that to the shilouette score
        sscore = metrics.silhouette_score(data, labels, metric='euclidean')
        score_ms = score_ms.append({'cluster_pred':'clust' + str(cntrs) + '_sample' + str(counter), 
                              'silhouette_score':sscore, 'quantile':n}, ignore_index=True)

score_ms.sort_values(by='silhouette_score', ascending=False)

The quantile of 0.4 calculated a high Silhouette score for sample 2 and generated 4 clusters.

**Spectral Clustering**

For the spectral clustering model, we use a range of clusters from 2 to 5, and we calculate the corresponding Silhouette scores for each.

In [0]:
# Initialize data frames
ypred_sc = pd.DataFrame()
score_sc = pd.DataFrame(columns=['cluster_pred','silhouette_score'])

# Keep track of counts of the models and use data from the different folds
for counter, data in enumerate([
    (X1, XPCA1),
    (X2, XPCA2),
    (X3, XPCA3),
    (X4, XPCA4)]):
    
    # Put the features into ypred.
    ypred_sc['pca_f1' + '_sample' + str(counter)] = data[1][:, 0]
    ypred_sc['pca_f2' + '_sample' + str(counter)] = data[1][:, 1]
    
    # Creating a list of possible number of clusters to test in kmeans.
    for nclust in range(2, 6):
        # Instantiating and fit_predicting model to then add to data frame
        sc = SpectralClustering(n_clusters=nclust)
        pred = sc.fit_predict(data[0])
        ypred_sc['clust' + str(nclust) + '_sample' + str(counter)] = pred
        # Calculating silhouette scores for the data and adding that to the shilouette score
        labels = sc.labels_
        sscore_sc = metrics.silhouette_score(data[0], labels, metric='euclidean')
        score_sc = score_sc.append({'cluster_pred':'clust' + str(nclust) + '_sample' + str(counter), 
                              'silhouette_score':sscore_sc}, ignore_index=True)

score_sc.sort_values(by='silhouette_score', ascending=False)

A 2 cluster configuration generates the highest silhouette score.

In [0]:
# For each  number of clusters, plot the clusters using the
# pca features for each sample.
for cluster in range(2, 6):
    
    # Make a grid of subplots.
    f, axarr = plt.subplots(2, 2)
    
    # Make a plot for each sample.
    for i in range(4):
        
        # PCA-created features.
        x_sub = ypred_sc['pca_f1_sample{}'.format(i)]
        y_sub = ypred_sc['pca_f2_sample{}'.format(i)]
        
        # Cluster assignments.
        c = ypred_sc['clust{}_sample{}'.format(cluster, i)]
        
        # Assign the subplot to its place on the grid.
        rows = int(np.floor(i / 2))
        cols = i % 2
        axarr[rows, cols].scatter(x_sub, y_sub, c=c)
        axarr[rows, cols].set_title('sample {}'.format(i))
        axarr[rows, cols].set_xlim([-.5, .5])
        axarr[rows, cols].set_ylim([-.5, 1.2])
    
    # Space out the plots so that the headings don't overlap axis values.
    plt.suptitle('{} Clusters'.format(cluster), fontsize=20)
    plt.tight_layout()
    plt.show()
    print('\n')

Watching the latter plots of the 2-feature PCA data, the 2 and 3 cluster configuration show a consistent solution across all samples. The 4 and 5 cluster configuration show overfitting for all samples.

**Affinity Propagation**

For the Affinity Propagation model, we allow the model to find the k number of cluster, and later we calculate the corresponding Silhouette scores for each.

In [0]:
# Initialize data frames
ypred = pd.DataFrame()
score_af = pd.DataFrame(columns=['cluster_pred','AF'])

# Keep track of counts of the models and use data from the different folds
for counter, data in enumerate([X1, X2, X3, X4]):
    # Ensuring all sets are the same lenght
    data = data[:4013][:]
    # Instantiating and fit_predicting model to then add to data frame
    af = AffinityPropagation().fit(data)
    cluster_centers_indices = af.cluster_centers_indices_
    n_clusters_ = len(cluster_centers_indices)
    #pred = af.fit_predict(data)
    #ypred['clust' + str(nclust) + '_sample' + str(counter)] = pred
    # Calculating silhouette scores for the data and adding that to the shilouette score
    labels = af.labels_
    sscore_af = metrics.silhouette_score(data, labels, metric='euclidean')
    score_af = score_af.append({'cluster_pred':'clust' + str(n_clusters_) + '_sample' + str(counter), 
                              'AF':sscore_af}, ignore_index=True)

score_af.sort_values(by='AF', ascending=False)

The number of clusters generated seem absurd, suggesting it isn't reliable.

In [0]:
# Calculate predicted values.
y_pred = KMeans(n_clusters=4, random_state=42).fit_predict(X_norm)

df_y = pd.DataFrame(y_pred)
df_y.columns = ['Cluster']

# Add the outcome back onto X
combined = X.join(df_y, how='inner')
combined.head()

In [0]:
# Create age buckets

def age_bucket(age):
    output = ''
    if age <=20:
        output = 'Under 20'
    elif (age > 20 and age <= 30):
        output = 'Between 20 and 30'
    elif (age > 30 and age <= 40):
        output = 'Between 30 and 40'
    elif (age > 40 and age <= 50):
        output = 'Between 40 and 50'
    elif (age > 50 and age <= 60):
        output = 'Between 50 and 60'
    else:
        output = 'Over 60'
    
    return output

In [0]:
combined['Age Bucket'] = combined['age'].apply(lambda x: age_bucket(x))

In [0]:
# Look at Gender Breakdown for Count
g = sns.factorplot(x='Age Bucket', col='Cluster', kind="count", data=combined, size=4)
g.set_xticklabels(rotation=90)

In [0]:
sns.factorplot(x='gender', col='Cluster', kind="count", data=combined, size=5)

Considering 0 = Female, 1 = Male, we can observe that in cluster 3 (best records), there is a predominant number of male runners. We confirm with following count values:

In [0]:
# Separate the clusters
cluster0 = combined[combined['Cluster']==0]
cluster1 = combined[combined['Cluster']==1]
cluster2 = combined[combined['Cluster']==2]
cluster3 = combined[combined['Cluster']==3]

In [0]:
# check Gender Breakdown
cluster0['gender'].value_counts()

In [0]:
cluster1['gender'].value_counts()

In [0]:
cluster2['gender'].value_counts()

In [0]:
cluster3['gender'].value_counts()

**K-prototype**




**Hopkins Statistics**

To understand if the dataset can be clustered, we used the Hopkins statistic, which tests the spatial randomness of the data and indicates the cluster tendency or how well the data can be clustered. It calculates the probability that a given data is generated by a uniform distribution (Alboukadel Kassambara, n.d.). The inference is as follows for a data of dimensions ‘d’:

- If the value is around 0.5 or lesser, the data is uniformly distributed and hence it is unlikely to have statistically significant clusters.

- If the value is between {0.7, ..., 0.99}, it has a high tendency to cluster and therefore likely to have statistically significant clusters.

In [0]:
#Hopkins Statistic is a way of measuring the cluster tendency of a data set.
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan
 
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) # heuristic from article [1]
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [0]:
#Use a random sample of Data for faster computation
data = data.sample(20000,random_state=41)
data.head()
#Resetting the indexs
data=data.reset_index(drop=True)

In [0]:
Num_features =data.select_dtypes(include=[np.number]).columns
hopkins(data[Num_features])

Result: This test is run on all the numerical variables of the entire dataset and the test statistic we got is 0.99 which indicates that data has a high tendency to cluster.

In [0]:
data.gender = data.gender.map(lambda x: 0 if x is 'F' else 1)
data.head()

In [0]:
#Selection of variables for PCA
Data_pca= data[['age', 'pace', 'division', 'overall', 'gender']]
print (Data_pca.dtypes)

In [0]:
#Principal Component
from sklearn.decomposition import PCA
pca = PCA(n_components=4, whiten=True)
Num_features=Data_pca.select_dtypes(include=[np.number]).columns
x=Data_pca[Num_features]
principalComponents = pca.fit_transform(x)

# Cumulative Explained Variance
cum_explained_var = []
for i in range(0, len(pca.explained_variance_ratio_)):
    if i == 0:
        cum_explained_var.append(pca.explained_variance_ratio_[i])
    else:
        cum_explained_var.append(pca.explained_variance_ratio_[i] + 
                                 cum_explained_var[i-1])

print(cum_explained_var)

In [0]:
#Principal Components converted to a Data frame
principalDf  = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])
principalDf.shape

In [0]:
#Concatenating the PCAs with the categorical variable
finalDf_Cat = pd.concat([principalDf, Data_pca['gender']], axis = 1)
finalDf_Cat.head(2)

In [0]:
!pip install kmodes

In [0]:
#Choosing optimal K value
from kmodes.kprototypes import KPrototypes

cost = []
X = finalDf_Cat
for num_clusters in list(range(2,7)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Huang', random_state=42,n_jobs=-2,max_iter=15,n_init=50) 
    kproto.fit_predict(X, categorical=[3])
    cost.append(kproto.cost_)

plt.plot(cost)
plt.xlabel('K')
plt.ylabel('cost')
plt.show

In [0]:
# Converting the dataset into matrix
X = finalDf_Cat.as_matrix()

In [0]:
# Running K-Prototype clustering
kproto = KPrototypes(n_clusters=2, init='Huang', verbose=0, random_state=42,max_iter=20, n_init=50,n_jobs=-2,gamma=.25) 
clusters = kproto.fit_predict(X, categorical=[3])

In [0]:
#Visualize K-Prototype clustering on the PCA projected Data
df=pd.DataFrame(finalDf_Cat)
df['Cluster_id']=clusters
print(df['Cluster_id'].value_counts())
sns.pairplot(df,hue='Cluster_id',palette='Dark2',diag_kind='kde')